# Importações

In [1]:
# Preparando ambiente (importando bibliotecas e downloads...)

!pip install nltk
import nltk
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
import re
import pandas as pd 
import numpy as np
import spacy
nlp = spacy.load('pt')

[nltk_data] Downloading package rslp to /home/karine/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Tratamento Dados Originais


In [2]:
"""
Logs_Original = open('logs/LogGeral_Original.txt', 'r', encoding='utf8')
Arquivo_Preprocessado = open('logs/LogGeral_PreProcessado.txt', 'w+')
"""

"\nLogs_Original = open('logs/LogGeral_Original.txt', 'r', encoding='utf8')\nArquivo_Preprocessado = open('logs/LogGeral_PreProcessado.txt', 'w+')\n"

In [3]:
"""
agente = ''
for i in Logs_Original:
    novo_agente = 'nulo'
    #print('oi')
    if (i[-11:] == '06/01/2020\n') or (i[-11:] == '06/02/2020\n'):
        novo_agente = i[:-11]

    if novo_agente == 'nulo':
        Arquivo_Preprocessado.writelines(str(agente + '\t' + i))
    else: 
        if novo_agente == 'AgAt1 - Juliano Storch':
            agente = 'AgAT1'
        elif novo_agente == 'AgAt2':
            agente = 'AgAT2'
        elif novo_agente == 'AgCD1 - Fiscal Cidadela':
            agente = 'AgCD1|Fiscal CD'
        elif novo_agente == 'AgCD2 / PrefeitoCD - GustavoLima':
            agente = 'AgCD2|Prefeito CD'
        elif novo_agente == 'AgCD3 | VerCD':
            agente = 'AgCD3|Vereador CD'
        elif novo_agente == 'EmpMaq - PrefeitaATLANTIS':
            agente = 'EmpMaq|Prefeito AT'
        elif novo_agente == 'EmpSem / Fiscal - Bruno':
            agente = 'EmpSem|Fiscal AT'  
"""

"\nagente = ''\nfor i in Logs_Original:\n    novo_agente = 'nulo'\n    #print('oi')\n    if (i[-11:] == '06/01/2020\n') or (i[-11:] == '06/02/2020\n'):\n        novo_agente = i[:-11]\n\n    if novo_agente == 'nulo':\n        Arquivo_Preprocessado.writelines(str(agente + '\t' + i))\n    else: \n        if novo_agente == 'AgAt1 - Juliano Storch':\n            agente = 'AgAT1'\n        elif novo_agente == 'AgAt2':\n            agente = 'AgAT2'\n        elif novo_agente == 'AgCD1 - Fiscal Cidadela':\n            agente = 'AgCD1|Fiscal CD'\n        elif novo_agente == 'AgCD2 / PrefeitoCD - GustavoLima':\n            agente = 'AgCD2|Prefeito CD'\n        elif novo_agente == 'AgCD3 | VerCD':\n            agente = 'AgCD3|Vereador CD'\n        elif novo_agente == 'EmpMaq - PrefeitaATLANTIS':\n            agente = 'EmpMaq|Prefeito AT'\n        elif novo_agente == 'EmpSem / Fiscal - Bruno':\n            agente = 'EmpSem|Fiscal AT'  \n"

In [4]:
"""
Logs_Original.close()
Arquivo_Preprocessado.close()
"""

'\nLogs_Original.close()\nArquivo_Preprocessado.close()\n'

# Carregamento e tratamento dos léxicos e datasets necessários

In [5]:
dicionario_lexico = {}
sentilexpt = open('SentiLex-lem-PT02.txt','r', encoding='utf8')
ontolp = open('lexico_v3.0.txt','r', encoding='utf8')

In [6]:
for i in sentilexpt:
    pos_ponto = i.find('.')
    palavra = (i[:pos_ponto])
    pol_pos = i.find('POL')
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')
    dicionario_lexico[palavra] = polaridade

In [7]:
for i in ontolp:
    split_dic = i.split(',')
    palavra = split_dic[0]
    if palavra not in dicionario_lexico:
        polaridade = split_dic[2]
        dicionario_lexico[palavra] = polaridade        

In [31]:
dic_sentimentos = {}
sentidic = open('dic_sentimento.txt','r', encoding='utf8')

In [32]:
for i in sentidic:
    split_dic = i.split('\t')
    dic_sentimentos[split_dic[0]] = split_dic[1][:-1]

In [8]:
degree_words = open('degree-words.txt','r', encoding='utf8')
degree_words_set = {}

In [9]:
for i in degree_words:
    split_dic = i.split(';')
    degree_words_set[split_dic[0]] = split_dic[1][:-1]

In [10]:
sentilexpt.close()
ontolp.close()
degree_words.close()
sentidic.close()

In [11]:
liwc = pd.read_csv("LIWC2007.txt",sep="\t",encoding="ISO-8859-1", names=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

In [12]:
liwc_set = {}
def cria_dic_liwc(x):
    lista = []
    if 126 in list(x):
        liwc_set[x.name] = 'pos'
    elif 127 in list(x):
        liwc_set[x.name] = 'neg'

In [13]:
liwc.apply(lambda x: cria_dic_liwc(x), axis=1)

a          None
aba        None
abafa      None
abafad*    None
abafada    None
           ... 
último     None
último*    None
út*        None
úteis      None
útil       None
Length: 127161, dtype: object

In [14]:
logs = pd.read_csv("logs/LogGeral_PreProcessado.txt", sep="\t" , encoding="utf8", names=['Agente', 'Frase'])

# Taxa de sentimento por frase

In [35]:
#df = pd.read_csv("dataset.csv", sep="," , encoding="utf8")
#df.drop(columns=['id', 'text_en', 'sentiment'], axis=1, inplace=True)
#df = df[:100]

lista_score_res = []

for frase in logs['Frase']:
    
    frase = nlp(frase)
    degree_words_head = []
    degree_words_aux = []
    score_frase = {'alegria':0, 'tristeza':0, 'raiva':0, 'medo':0, 'nojo':0}
    
    for token in frase:
        if(token.text in degree_words_set):
            degree_words_head.append(token.head.text)
            degree_words_aux.append(int(degree_words_set.get(token.text)))
            
    for token in frase:
        if(token.text in dic_sentimentos):
            dic = dic_sentimentos.get(token.text)
            neg = 0
            if token.head.text in degree_words_head:                
                neg = int(degree_words_aux[degree_words_head.index(token.head.text)])
            score_frase[dic] = int(score_frase.get(dic)) + 1 + neg

    score_resultado = ('neutro', 0)
    for sentimento, valor in score_frase.items():
        s, v = score_resultado
        if abs(valor) > v:
            score_resultado = (sentimento, valor)
        
    s, v = score_resultado
    lista_score_res.append(s)

In [36]:
logs['Sentimento'] = lista_score_res

In [39]:
logs['Sentimento'].value_counts()

neutro      826
alegria     107
raiva        44
tristeza     19
Name: Sentimento, dtype: int64

# Treinamento do Modelo

## Pre-processamento

In [17]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [40]:
def removeStopWords():
    frases = []
    for (palavras, sentimento) in zip(logs.Frase, logs.Sentimento):
        semStop = [ p for p in palavras.split() if p not in stop_words]
        frases.append((semStop, sentimento))
    return frases

In [41]:
def realiza_stem(dados):
    stemmer = nltk.stem.RSLPStemmer()
    frases_sem_Stemming = []
    for (palavras, sentimento) in dados:
        com_Stemming = [str(stemmer.stem(p)) for p in palavras.split() if p not in stop_words]
        frases_sem_Stemming.append((com_Stemming, sentimento))
    return pd.DataFrame(frases_sem_Stemming)

In [42]:
def preprocessamento():
    sem_stopwords = removeStopWords()    
    return realiza_stem(sem_stop_words)

## Treinamento

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1, 1))
vect.fit(logs.Frase)
text_vect = vect.transform(logs.Frase)

In [44]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(text_vect, logs.Sentimento, test_size = 0.3, random_state = 42)

In [45]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='newton-cg')
#y=y.astype('int')
clf = clf.fit(X_train, y_train)

In [46]:
from sklearn.metrics import f1_score
y_prediction = clf.predict(X_test)

f1 = f1_score(y_prediction, y_test, average='weighted')

print(f1)

0.925969261142232
